In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6245575155571702763
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7705861831858673009
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=300

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'word2vec-google-news-300')
    print(len(embeddings_index.keys()))
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

18515


In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.421088668264293


In [12]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [13]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 8s - loss: 0.7160 - acc: 0.5963 - val_loss: 0.5918 - val_acc: 0.6487
Epoch 2/100
 - 4s - loss: 0.4182 - acc: 0.8176 - val_loss: 0.5602 - val_acc: 0.6611
Epoch 3/100
 - 4s - loss: 0.2230 - acc: 0.9333 - val_loss: 0.5867 - val_acc: 0.6553
Epoch 4/100
 - 4s - loss: 0.1162 - acc: 0.9739 - val_loss: 0.6527 - val_acc: 0.6689
Epoch 5/100
 - 4s - loss: 0.0673 - acc: 0.9862 - val_loss: 0.7096 - val_acc: 0.6701
Epoch 6/100
 - 4s - loss: 0.0574 - acc: 0.9844 - val_loss: 0.7387 - val_acc: 0.6638
Epoch 7/100
 - 4s - loss: 0.0467 - acc: 0.9880 - val_loss: 0.7647 - val_acc: 0.6656
Epoch 8/100
 - 4s - loss: 0.0409 - acc: 0.9883 - val_loss: 0.8167 - val_acc: 0.6674
Epoch 9/100
 - 4s - loss: 0.0551 - acc: 0.985

Epoch 86/100
 - 5s - loss: 0.0556 - acc: 0.9913 - val_loss: 1.8188 - val_acc: 0.6692
Epoch 87/100
 - 5s - loss: 0.0495 - acc: 0.9916 - val_loss: 1.6691 - val_acc: 0.6629
Epoch 88/100
 - 5s - loss: 0.0538 - acc: 0.9922 - val_loss: 1.8680 - val_acc: 0.6743
Epoch 89/100
 - 5s - loss: 0.0625 - acc: 0.9883 - val_loss: 2.3091 - val_acc: 0.6376
Epoch 90/100
 - 5s - loss: 0.0865 - acc: 0.9829 - val_loss: 1.7059 - val_acc: 0.6662
Epoch 91/100
 - 5s - loss: 0.0589 - acc: 0.9862 - val_loss: 2.0593 - val_acc: 0.6680
Epoch 92/100
 - 5s - loss: 0.0623 - acc: 0.9850 - val_loss: 1.6791 - val_acc: 0.6641
Epoch 93/100
 - 5s - loss: 0.0346 - acc: 0.9910 - val_loss: 1.8794 - val_acc: 0.6656
Epoch 94/100
 - 5s - loss: 0.0582 - acc: 0.9892 - val_loss: 1.7962 - val_acc: 0.6689
Epoch 95/100
 - 5s - loss: 0.0597 - acc: 0.9901 - val_loss: 1.8929 - val_acc: 0.6653
Epoch 96/100
 - 5s - loss: 0.0625 - acc: 0.9904 - val_loss: 1.9232 - val_acc: 0.6641
Epoch 97/100
 - 4s - loss: 0.0608 - acc: 0.9916 - val_loss: 1.918

In [14]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 9s - loss: 0.7259 - acc: 0.5807 - val_loss: 0.5991 - val_acc: 0.6361
Epoch 2/100
 - 5s - loss: 0.4601 - acc: 0.7728 - val_loss: 0.5665 - val_acc: 0.6602
Epoch 3/100
 - 5s - loss: 0.2129 - acc: 0.9237 - val_loss: 0.6514 - val_acc: 0.6668
Epoch 4/100
 - 5s - loss: 0.0872 - acc: 0.9790 - val_loss: 0.7223 - val_acc: 0.6593
Epoch 5/100
 - 5s - loss: 0.0734 - acc: 0.9826 - val_loss: 0.7258 - val_acc: 0.6686
Epoch 6/100
 - 5s - loss: 0.0640 - acc: 0.9820 - val_loss: 0.9573 - val_acc: 0.6749
Epoch 7/100
 - 5s - loss: 0.0554 - acc: 0.9883 - val_loss: 0.9523 - val_acc: 0.6653
Epoch 8/100
 - 5s - loss: 0.0553 - acc: 0.9883 - val_loss: 0.8485 - val_acc: 0.6626
Epoch 9/100
 - 5s - loss: 0.0481 - acc: 0.9889 - val_loss: 0.8501 - val_acc: 0.6620
Epoch 10/100
 - 5s - loss: 0.0380 - acc: 0.9889 - val_loss: 0.9182 - val_acc: 0.6716
Epoch 11/100
 - 5s - loss: 0.0467 - acc: 0.9841 - val_loss: 0.9470 - val_acc: 0.6716
Epoch 12/100
 - 5s - loss:

Epoch 97/100
 - 5s - loss: 0.0354 - acc: 0.9913 - val_loss: 1.8827 - val_acc: 0.6680
Epoch 98/100
 - 5s - loss: 0.0415 - acc: 0.9916 - val_loss: 1.9227 - val_acc: 0.6707
Epoch 99/100
 - 5s - loss: 0.0393 - acc: 0.9916 - val_loss: 2.0684 - val_acc: 0.6692
Epoch 100/100
 - 5s - loss: 0.0400 - acc: 0.9925 - val_loss: 2.1013 - val_acc: 0.6707
0.6162464985994398


In [16]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.6765 - acc: 0.5669 - val_loss: 0.6555 - val_acc: 0.6244
Epoch 2/100
 - 2s - loss: 0.5871 - acc: 0.6853 - val_loss: 0.5951 - val_acc: 0.6454
Epoch 3/100
 - 2s - loss: 0.4237 - acc: 0.8055 - val_loss: 0.6014 - val_acc: 0.6439
Epoch 4/100
 - 2s - loss: 0.2400 - acc: 0.9170 - val_loss: 0.6632 - val_acc: 0.6584
Epoch 5/100
 - 2s - loss: 0.1473 - acc: 0.9621 - val_loss: 0.7674 - val_acc: 0.6481
Epoch 6/100
 - 2s - loss: 0.0875 - acc: 0.9814 - val_loss: 0.7919 - val_acc: 0.6526
Epoch 7/100
 - 2s - loss: 0.0787 - acc: 0.9820 - val_loss: 0.8693 - val_acc: 0.6505
Epoch 8/100
 - 2s - loss: 0.0709 - acc: 0.9868 - val_loss: 0.9073 - val_acc: 0.6614
Epoch 9/100
 - 2s - loss: 0.0647 - acc: 0.9874 - val_loss: 0.8888 - val_acc: 0.6523
Epoch 10/100
 - 2s - loss: 0.0613 - acc: 0.9859 - val_loss: 0.9134 - val_acc: 0.6626
Epoch 11/100
 - 2s - loss: 0.0520

Epoch 94/100
 - 2s - loss: 0.0154 - acc: 0.9925 - val_loss: 2.4553 - val_acc: 0.6535
Epoch 95/100
 - 2s - loss: 0.0146 - acc: 0.9928 - val_loss: 2.4344 - val_acc: 0.6499
Epoch 96/100
 - 2s - loss: 0.0141 - acc: 0.9928 - val_loss: 2.5470 - val_acc: 0.6523
Epoch 97/100
 - 2s - loss: 0.0143 - acc: 0.9910 - val_loss: 2.5707 - val_acc: 0.6535
Epoch 98/100
 - 2s - loss: 0.0136 - acc: 0.9916 - val_loss: 2.7235 - val_acc: 0.6526
Epoch 99/100
 - 2s - loss: 0.0148 - acc: 0.9925 - val_loss: 2.5563 - val_acc: 0.6532
Epoch 100/100
 - 2s - loss: 0.0135 - acc: 0.9913 - val_loss: 2.4854 - val_acc: 0.6538
0.6144578313253012


In [18]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 4s - loss: 0.6509 - acc: 0.5963 - val_loss: 0.6139 - val_acc: 0.6343
Epoch 2/100
 - 3s - loss: 0.4205 - acc: 0.8073 - val_loss: 0.5954 - val_acc: 0.6572
Epoch 3/100
 - 3s - loss: 0.1655 - acc: 0.9504 - val_loss: 0.6874 - val_acc: 0.6487
Epoch 4/100
 - 3s - loss: 0.0858 - acc: 0.9802 - val_loss: 0.7523 - val_acc: 0.6644
Epoch 5/100
 - 3s - loss: 0.0624 - acc: 0.9871 - val_loss: 0.8481 - val_acc: 0.6668
Epoch 6/100
 - 3s - loss: 0.0599 - acc: 0.9859 - val_loss: 0.7555 - val_acc: 0.6559
Epoch 7/100
 - 3s - loss: 0.0432 - acc: 0.9874 - val_loss: 0.8018 - val_acc: 0.6523
Epoch 8/100
 - 3s - loss: 0.0320 - acc: 0.9871 - val_loss: 0.8586 - val_acc: 0.6602
Epoch 9/100
 - 3s - loss: 0.0254 - acc: 0.9877 - val_loss: 0.9216 - val_acc: 0.6581
Epoch 10/100
 - 4s - loss: 0.0210 - acc: 0.9886 - val_loss: 0.8987 - val_acc: 0.6520
Epoch 11/100
 - 3s - loss: 0.0205 - acc: 0.9889 - val_loss: 0.9439 - val_acc: 0.6487
Epoch 12/100
 - 4s - loss:

Epoch 97/100
 - 3s - loss: 0.0137 - acc: 0.9946 - val_loss: 2.1982 - val_acc: 0.6439
Epoch 98/100
 - 3s - loss: 0.0139 - acc: 0.9919 - val_loss: 2.2909 - val_acc: 0.6436
Epoch 99/100
 - 3s - loss: 0.0134 - acc: 0.9916 - val_loss: 2.2828 - val_acc: 0.6451
Epoch 100/100
 - 3s - loss: 0.0144 - acc: 0.9913 - val_loss: 2.3824 - val_acc: 0.6490
0.620039037085231


In [20]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 723s - loss: 0.6825 - acc: 0.5681 - val_loss: 0.6767 - val_acc: 0.5730
Epoch 2/10
 - 824s - loss: 0.6527 - acc: 0.6060 - val_loss: 0.6669 - val_acc: 0.5898
Epoch 3/10
 - 830s - loss: 0.6173 - acc: 0.6459 - val_loss: 0.6611 - val_acc: 0.5931
Epoch 4/10
 - 784s - loss: 0.5751 - acc: 0.6907 - val_loss: 0.6543 - val_acc: 0.6205
Epoch 5/10
 - 714s - loss: 0.5227 - acc: 0.7304 - val_loss: 0.6722 - val_acc: 0.6172
Epoch 6/10
 - 709s - loss: 0.4708 - acc: 0.7683 - val_loss: 0.7221 - val_acc: 0.6031
Epoch 7/10
 - 710s - loss: 0.4036 - acc: 0.8091 - val_loss: 0.7398 - val_acc: 0.6358
Epoch 8/10
 - 714s - loss: 0.3488 - acc: 0.8389 - val_loss: 0.7850 - val_acc: 0.6376
Epoch 9/10
 - 714s - loss: 0.2902 - acc: 0.8786 - val_loss: 0.8883 - val_acc: 0.6367
Epoch 10/10
 - 714s - loss: 0.2184 - acc: 0.9167 - val_loss: 0.9495 - val_acc: 0.6448
0.6014834794335805


In [22]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 752s - loss: 0.6761 - acc: 0.5726 - val_loss: 0.6570 - val_acc: 0.6088
Epoch 2/10
 - 751s - loss: 0.5644 - acc: 0.7048 - val_loss: 0.6276 - val_acc: 0.6346
Epoch 3/10
 - 756s - loss: 0.3948 - acc: 0.8227 - val_loss: 0.7626 - val_acc: 0.6526
Epoch 4/10
 - 754s - loss: 0.2605 - acc: 0.8924 - val_loss: 0.9046 - val_acc: 0.6481
Epoch 5/10
 - 754s - loss: 0.1629 - acc: 0.9387 - val_loss: 1.0046 - val_acc: 0.6599
Epoch 6/10
 - 749s - loss: 0.1206 - acc: 0.9501 - val_loss: 1.2467 - val_acc: 0.6496
Epoch 7/10
 - 761s - loss: 0.0738 - acc: 0.9723 - val_loss: 1.4522 - val_acc: 0.6466
Epoch 8/10
 - 785s - loss: 0.0528 - acc: 0.9796 - val_loss: 1.5676 - val_acc: 0.6448
Epoch 9/10
 - 879s - loss: 0.0508 - acc: 0.9760 - val_loss: 1.6209 - val_acc: 0.6520
Epoch 10/10
 - 761s - loss: 0.0421 - acc: 0.9820 - val_loss: 1.9230 - val_acc: 0.6475
0.6286799620132953
